In [1]:
%load_ext autoreload
%autoreload 2
%connect_info
%pprint 1
# General imports
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from typing import NamedTuple, List,Tuple
from dataclasses import dataclass
from gt.solutions.br import BrownRobinsonOptimizer
from IPython.display import display

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

{
  "shell_port": 64569,
  "iopub_port": 64571,
  "stdin_port": 64573,
  "control_port": 64576,
  "hb_port": 64579,
  "ip": "127.0.0.1",
  "key": "d0551bb2-e3057811c6c4230f5be764aa",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-17aad4ac-2972-42b5-9845-ce7939879c54.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
Pretty printing has been turned OFF


10:14:50 DEBUG:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [45]:

@dataclass
class AntagonisticGameOptimizer:
    A: float
    B: float
    c: float
    d: float
    e: float

    def H(self, x: float, y: float) -> float:
        return self.A*x**2 + self.B*y**2 + self.c*x*y + self.d*x + self.e*y
    
    def __post_init__(self):
        if 2*self.A >= 0 and 2*self.B <= 0:
            raise ValueError("This game is not solvable by this solver")
        
    def analytical_solution(self) -> Tuple[float, float, float]:
        x = (self.c*self.e - 2*self.B*self.d) / (4*self.B*self.A - self.c**2)
        y = -(self.c*x + self.e) / (2*self.B)
        x = x if x <= -self.e/self.c else 0
        y = y if y >= -self.d/self.c else 0
        solution = self.H(x, y)
        print(f"Analytical optimal solution: x={x:.3f}, y={y:.3f}, H(x,y)={solution:.2f}")
        return x,y,solution
    
    @staticmethod
    def minmax(matrix) -> Tuple[int, int]:
        max_points = np.amax(matrix, axis=0)
        column = np.argmin(max_points)
        row = np.argmax(matrix[:,column])
        #print(f"Minimax position: {row, column}")
        return row, column
        
    @staticmethod
    def maxmin(matrix) -> Tuple[int, int]:
        min_points = np.amin(matrix, axis=1)
        #print(f"Maxmin points: {min_points}")
        row = np.argmax(min_points)
        column = np.argmin(matrix[row])
        #print(f"Maximin position: {row, column}")
        return row, column
    
    @classmethod
    def has_point(cls, matrix) -> bool:
        return cls.maxmin(matrix) == cls.minmax(matrix)
        
    def step(self, matrix_size: int) -> Tuple[float, float, float]:
        M = np.empty([matrix_size + 1, matrix_size + 1])
        for i in range(0, matrix_size + 1):
            for j in range(0, matrix_size + 1):
                M[i][j] = self.H(float(i) / matrix_size, float(j) / matrix_size)
        df = pd.DataFrame(M)
        #print(f"M table with N={matrix_size}")
        #display(df)
        if not self.has_point(M):
            print("No saddle point. Use Brown-Robinson for decisions...")
            opt = BrownRobinsonOptimizer(M, max_steps=10000, verbose=False)
            strategies, _ = opt.fit(0.001)
            x = strategies.A
            y = strategies.B
            H = x @ M @ y.T
            #print(f"X: {x}, Y: {y}")
            x = np.argmax(x) / matrix_size
            y = np.argmax(y) / matrix_size
        else:
            mins = np.amin(M, axis=1)
            #print(f"Minima for M: {mins}")
            mins_pos = np.argmin(M, axis=1)
            #print(f"Minima positions for M: {mins_pos}")
            H = max_min = np.amax(mins)
            max_min_pos = np.argmax(mins)
            #print(f"Maximin positions for M: {max_min_pos}")
            y = mins_pos[max_min_pos] / matrix_size
            x = max_min_pos / matrix_size

        print(f"N: {matrix_size},\tx={x:.2f},\ty={y:.2f},\tH={H:.2f}")
        return x,y,H
    
    def fit(self, iterations: int=10) -> Tuple[float, float, float]:
        states = []
        for N in range(2, iterations+1):
            states.append(self.step(N))
            print("============")
        states = sorted(states, key=lambda x: x[2], reverse=True)
        print(f"Best solution: x={states[0][0]:.3f}, y={states[0][1]:.3f}, H={states[0][2]:.3f}")
        return states[0]
        

In [46]:
opt = AntagonisticGameOptimizer(A=-3, B=3/2, c=18/5, d=-18/50, e=-72/25)
#opt = AntagonisticGameOptimizer(A=-5, B=9/2, c=15, d=-9/2, e=-9)
opt = AntagonisticGameOptimizer(A=-6, B=32/5, c=16, d=-16/5, e=-64/5)
opt.fit(iterations=10)
opt.analytical_solution()

N: 2,	x=0.50,	y=0.50,	H=-3.90
No saddle point. Use Brown-Robinson for decisions...
N: 3,	x=0.33,	y=0.67,	H=-3.77
N: 4,	x=0.50,	y=0.50,	H=-3.90
No saddle point. Use Brown-Robinson for decisions...
N: 5,	x=0.40,	y=0.60,	H=-3.78
N: 6,	x=0.33,	y=0.50,	H=-3.87
No saddle point. Use Brown-Robinson for decisions...
N: 7,	x=0.43,	y=0.43,	H=-3.83
N: 8,	x=0.38,	y=0.50,	H=-3.84
No saddle point. Use Brown-Robinson for decisions...
N: 9,	x=0.44,	y=0.44,	H=-3.84
N: 10,	x=0.40,	y=0.50,	H=-3.84
Best solution: x=0.333, y=0.667, H=-3.769
Analytical optimal solution: x=0.400, y=0.500, H(x,y)=-3.84


(0.39999999999999997, 0.5000000000000001, -3.8400000000000007)